<img src="https://imagedelivery.net/Dr98IMl5gQ9tPkFM5JRcng/3e5f6fbd-9bc6-4aa1-368e-e8bb1d6ca100/Ultra" alt="Image description" width="160" />

<br/>

# Introduction to Contextual AI Platform

The Contextual APIs provide a simple interface to our state-of-the-art Contextual Language Models (CLMs). Use this guide to learn the basics of how to create your first agent programmatically. In this demo, we will be creating a RAG application for financial documents.

To run this notebook interactively, you can open it in Google Colab:

<a target="_blank" href="https://colab.research.google.com/github/ContextualAI/ContextualAI-Examples/blob/main/python/platform-api-example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Setup of Libraries, API, Dataset

To begin, you will need an API key to securely access the API. Please contact Contextual's sales team to get your API key.

In [46]:
# TODO: Set up your API key here
CONTEXTUAL_API_KEY="key-..."

In [12]:
import os
import requests
from contextual import ContextualAI

# create a client
client = ContextualAI(
    api_key=CONTEXTUAL_API_KEY,
)

# test the API Key
try:
    response = create_agent_output = client.agents.list()
    print("Valid API Key.")
except Exception as e:
    print(f"Invalid API Key: {e}")

Valid API Key.


## Step 1: Create your Datastore


You will need to first create a datastore for your Agent using the  `/datastores` endpoint.  Each Agent will have it's own datastore where the Agent data will be stored securely.

In [47]:
# Create an datastore with name 'new_demo_datastore'
try:
    create_datastore_output = client.datastores.create(
        name="new_demo_datastore"
    )
    print(create_datastore_output.model_dump_json())
    datastore_id = create_datastore_output.id
except Exception as e:
    print(f"Encountered error: {e}")

{"id":"b9d8c4e2-9a79-42e2-834f-3d9165930724"}


## Step 2: Ingest documents into your datastore

You can now ingest documents into your Application's datastore using the `/datastores` endpoint. We've provided a sample document for you to use (Apple's 2023Q1 10-Q filing). You can also use your own documents here.

In [48]:
# Function to download data from Google Drive
def download_google_drive_pdf(url: str, output_path: str):
   """
   Download a PDF from Google Drive using the file ID
   """
   response = requests.get(url)
   response.raise_for_status()
   with open(output_path, 'wb') as f:
       f.write(response.content)
   print(f"Downloaded {output_path}")
   return str(output_path)

In [49]:
# Download sample data from Google Drive
file_path = download_google_drive_pdf(
    "https://drive.google.com/uc?export=download&id=1k4C-o7mLEmthbeyhccNEjYrQb10UFOfT",
    "2023_Q1_APPLE.pdf"
)

Downloaded 2023_Q1_APPLE.pdf


In [51]:
#  Upload a local file to Contextual AI datastore
try:
    with open('2023_Q1_APPLE.pdf', 'rb') as f:
        output = client.datastores.documents.ingest(
            datastore_id=datastore_id,
            file=f
        )
        print(output.model_dump_json())
        document_id = output.id
except Exception as e:
    print(f"Encountered error: {e}")


{"id":"ea7b06f6-f951-4278-b446-760b9dd7a032"}


In [78]:
# track the status of the datastore ingest
client.datastores.documents.metadata(
    datastore_id=datastore_id,
    document_id=document_id,
)

DocumentMetadata(id='ea7b06f6-f951-4278-b446-760b9dd7a032', name='2023_Q1_APPLE.pdf', status='completed')

## Step 3: Create your Agent

Now, you can use the `/agents` endpoint to create your Agent and link it to the datastore.

First we'll create a sample prompt:

In [74]:
# Sample system guideline
sample_guideline = '''
You are an AI assistant specialized in financial analysis and reporting. Your responses should be precise, accurate, and sourced exclusively from official financial documentation provided to you. Please follow these guidelines:

Data Analysis & Response Quality:
* Only use information explicitly stated in provided documentation (e.g., earnings releases, financial statements, investor presentations)
* Present comparative analyses using structured formats with tables and bullet points where appropriate
* Include specific period-over-period comparisons (quarter-over-quarter, year-over-year) when relevant
* Maintain consistency in numerical presentations (e.g., consistent units, decimal places)
* Flag any one-time items or special charges that impact comparability

Technical Accuracy:
* Use industry-standard financial terminology
* Define specialized acronyms on first use
* Never interchange distinct financial terms (e.g., revenue, profit, income, cash flow)
* Always include units with numerical values
* Pay attention to fiscal vs. calendar year distinctions
* Present monetary values with appropriate scale (millions/billions)

Response Format:
* Begin with a high-level summary of key findings when analyzing data
* Structure detailed analyses in clear, hierarchical formats
* Use markdown for lists, tables, and emphasized text
* Maintain a professional, analytical tone
* Present quantitative data in consistent formats (e.g., basis points for ratios)

Critical Guidelines:
* Do not make forward-looking projections unless directly quoted from source materials
* Avoid opinions, speculation, or assumptions
* If information is unavailable or irrelevant, clearly state this without additional commentary
* Answer questions directly, then stop
* Do not reference source document names or file types in responses
* Focus only on information that directly answers the query

For any analysis, provide comprehensive insights using all relevant available information while maintaining strict adherence to these guidelines and focusing on delivering clear, actionable information.
'''


Next we'll create the agent with our sample prompt, datastore, and a name and description:

In [76]:
# Create an agent with name 'Financial Analysis RAG'
try:
    create_agent_output = client.agents.create(
        name="Financial Analysis RAG",
        description="Application for analyzing the SEC filings of major companies",
        system_prompt=sample_guideline,
        datastore_ids=[datastore_id]
    )
    print(create_agent_output.model_dump_json())
    agent_id = create_agent_output.id
except Exception as e:
    print(f"Encountered error: {e}")

{"id":"66e663fc-a96f-4378-9842-77bcd7b89ce6","datastore_ids":["621134eb-ab0b-4973-a11f-d8a6113093cd"]}


## Step 4: Query your Application

Finally, let's query our application to see if its working.

**Note:** It may take a few minutes for the document to be ingested and processed.

In [77]:

try:
    query = client.agents.query.create(
        agent_id=agent_id,
        messages=[{
            # Input your question here
            "content": "What is Apple Inc's revenue?",
            "role": "user",
        }]
    )
    print(query.message.content)
except Exception as e:
    print(f"Encountered error: {e}")

**Summary:**

Apple Inc.'s revenue for the three months ended December 31, 2022, was $117,154 million, a 5% decrease from $123,945 million in the same period last year.[1]()[3]()

**Revenue Breakdown:**

The following table shows the revenue breakdown by category for the three months ended December 31, 2022, and December 25, 2021:

| Category | Three Months Ended December 31, 2022 | Three Months Ended December 25, 2021 | Change |
| --- | --- | --- | --- |
| iPhone | $65,775 million | $71,628 million | (8)% |[1]()[3]()
| Mac | $7,735 million | $10,852 million | (29)% |[1]()[3]()
| iPad | $9,396 million | $7,248 million | 30% |[3]()
| Wearables, Home and Accessories | $13,482 million | $14,701 million | (8)% |[1]()[3]()
| Services | $20,766 million | $19,516 million | 6% |[3]()
| Total Net Sales | $117,154 million | $123,945 million | (5)% |[1]()[3]()

Note: The revenue figures are in millions of US dollars.[1]()[2]()[3]()


In [79]:
try:
    query = client.agents.query.create(
        agent_id=agent_id,
        messages=[{
            # Input your question here
            "content": "What is Apple's net income in billions (not millions)?",
            "role": "user",
        }]
    )
    print(query.message.content)
except Exception as e:
    print(f"Encountered error: {e}")

Apple's net income for the three months ended December 31, 2022, is $29.998 billion.[1]()[2]()


In [80]:
try:
    query = client.agents.query.create(
        agent_id=agent_id,
        messages=[{
            # Input your question here
            "content": "What is the value of Apple's non-current assets?",
            "role": "user",
        }]
    )
    print(query.message.content)
except Exception as e:
    print(f"Encountered error: {e}")

The value of Apple's non-current assets as of December 31, 2022, is $217,970 million.[1]() This consists of:

* Marketable securities: $114,095 million[1]()[2]()
* Property, plant, and equipment, net: $42,951 million[1]()
* Other non-current assets: $60,924 million[1]()


## Next Steps

In this Notebook, we've created a simple RAG agent in the finance domain. Our APIs offer many more capabilities than what is shown in this Notebook, including the ability to tune and evaluate your RAG system. You can learn more at [docs.contextual.ai](https://docs.contextual.ai/)